In [1]:
import polars as pl
import numpy as np
import os

In [2]:
data_dir = "/Users/maxwoolf/Downloads"

df = pl.read_parquet(
    os.path.join(data_dir, "movie_data_plus_embeds_all.parquet"),
).with_columns(
    title=pl.col("json").str.json_path_match(r"$.title"),
)

df

tconst,startYear,numVotes,averageRating,json,embedding,title
str,i64,i64,f64,str,"array[f32, 768]",str
"""tt0000009""",1894,224,5.4,"""{ ""title"": ""Miss Jerry"", ""…","[-0.007815, -0.022642, … 0.005391]","""Miss Jerry"""
"""tt0000147""",1897,558,5.3,"""{ ""title"": ""The Corbett-Fitz…","[0.012021, 0.014255, … -0.015754]","""The Corbett-Fitzsimmons Fight"""
"""tt0000574""",1906,985,6.0,"""{ ""title"": ""The Story of the…","[-0.010052, -0.015825, … 0.040161]","""The Story of the Kelly Gang"""
"""tt0000591""",1907,31,5.6,"""{ ""title"": ""The Prodigal Son…","[0.00765, 0.019661, … -0.010763]","""The Prodigal Son"""
"""tt0000630""",1908,33,3.2,"""{ ""title"": ""Hamlet"", ""genr…","[0.03492, 0.00301, … 0.027586]","""Hamlet"""
…,…,…,…,…,…,…
"""tt9915790""",2019,45,7.0,"""{ ""title"": ""Bobbyr Bondhura""…","[-0.008241, -0.024547, … -0.014563]","""Bobbyr Bondhura"""
"""tt9916160""",2019,52,6.2,"""{ ""title"": ""Drømmeland"", ""…","[-0.014737, -0.035892, … 0.027569]","""Drømmeland"""
"""tt9916190""",2020,263,3.6,"""{ ""title"": ""Safeguard"", ""g…","[0.014303, -0.018036, … -0.008043]","""Safeguard"""


In [3]:
embeddings = df["embedding"].to_numpy()
embeddings.shape

(242552, 768)

Test Star Wars


In [4]:
movie = df.filter(pl.col("tconst") == "tt0076759").rows(named="true")[0]
query_embed = np.array(movie["embedding"])
query_embed.shape

(768,)

In [5]:
cossims = query_embed @ embeddings.T
cossims[:10]

array([0.70110115, 0.70786815, 0.74661582, 0.70941803, 0.74154506,
       0.68520667, 0.75312955, 0.69515846, 0.73177594, 0.74492537])

In [6]:
# https://stackoverflow.com/a/6910672
top_idx = np.argsort(cossims)[-100:][::-1]
top_idx

array([ 35035,  37464,  40931,  60563, 181973, 181974,  60405, 181471,
       109964,  91007,  90980, 132629,  42606, 111311,  60562, 203993,
       176142, 216136, 127845, 103395,  83506,  41509, 204244,  96999,
       201143, 122918, 133060,  74768, 120547, 223960,  41746, 117674,
       217786, 216352, 132349, 173229, 232886,  41664, 102192,  32698,
        90152, 109512, 180045,  28415, 213537, 130224, 169961,  94195,
       122994, 167597,  38975,  55995,  34861, 234314, 123386,  32495,
       146259, 100938, 201460,  79438,  66278,  71150, 224144,  50756,
       134354,  33617,  94458, 136133,  89154, 153715,  38962, 122317,
       140051, 107890, 130848,  89396,  42344,  42647,  77778, 169389,
        42856,  59467, 126664,  58857,  59074, 130494,  30926, 137828,
        88594, 198562,  34263,  31740, 100845,  28084,  56695,  50288,
        37148, 163303,  76346, 126490])

In [7]:
top_cossims = cossims[top_idx]
top_cossims

array([0.99999988, 0.92043304, 0.91696225, 0.90745466, 0.90449376,
       0.90331304, 0.89827816, 0.89408195, 0.89119305, 0.89097709,
       0.89063769, 0.88878801, 0.8843457 , 0.88232312, 0.88046529,
       0.87946889, 0.87796033, 0.87576742, 0.87423456, 0.87414255,
       0.87391586, 0.87042162, 0.86945008, 0.86812902, 0.86658614,
       0.86582452, 0.86331534, 0.86064976, 0.8588337 , 0.85881803,
       0.85645084, 0.85576067, 0.85383235, 0.85376825, 0.85354023,
       0.85345668, 0.85337922, 0.85278556, 0.85221014, 0.85175825,
       0.85113342, 0.84912398, 0.84840477, 0.847578  , 0.84756099,
       0.84717688, 0.84621708, 0.84608658, 0.84575343, 0.84563859,
       0.84543791, 0.84488446, 0.84461896, 0.84330353, 0.84317802,
       0.84310937, 0.8427787 , 0.84273456, 0.84273263, 0.84170224,
       0.84149097, 0.84105725, 0.84096003, 0.84085799, 0.84082539,
       0.84033219, 0.84010948, 0.83957534, 0.83865016, 0.83844608,
       0.83727246, 0.83722156, 0.83681763, 0.83618749, 0.83600

In [8]:
df_related = (
    df[top_idx]
    .select(["tconst", "startYear", "title"])
    .with_columns(cossim=top_cossims)
)
df_related

tconst,startYear,title,cossim
str,i64,str,f64
"""tt0076759""",1977,"""Star Wars: Episode IV - A New …",1.0
"""tt0080684""",1980,"""Star Wars: Episode V - The Emp…",0.920433
"""tt0086190""",1983,"""Star Wars: Episode VI - Return…",0.916962
"""tt0121766""",2005,"""Star Wars: Episode III - Reven…",0.907455
"""tt2527336""",2017,"""Star Wars: Episode VIII - The …",0.904494
…,…,…,…
"""tt0102057""",1991,"""Hook""",0.830747
"""tt0080137""",1975,"""Fighter Pilots""",0.830247
"""tt1835948""",2010,"""Edge of the Empire""",0.82998


In [9]:
df_md = df_related.head(10).select(
    pl.format(
        "[{} ({})](https://www.imdb.com/title/{}/)", "title", "startYear", "tconst"
    ).alias("title"),
    pl.col("cossim").round(3),
)

with pl.Config() as cfg:
    cfg.set_tbl_formatting("ASCII_MARKDOWN")
    cfg.set_fmt_str_lengths(10**5)
    cfg.set_tbl_width_chars(-1)
    cfg.set_tbl_hide_column_data_types(True)
    cfg.set_tbl_hide_dataframe_shape(True)
    cfg.set_trim_decimal_zeros(False)
    print(df_md)


| title                                                                                          | cossim |
|------------------------------------------------------------------------------------------------|--------|
| [Star Wars: Episode IV - A New Hope (1977)](https://www.imdb.com/title/tt0076759/)             | 1.0    |
| [Star Wars: Episode V - The Empire Strikes Back (1980)](https://www.imdb.com/title/tt0080684/) | 0.92   |
| [Star Wars: Episode VI - Return of the Jedi (1983)](https://www.imdb.com/title/tt0086190/)     | 0.917  |
| [Star Wars: Episode III - Revenge of the Sith (2005)](https://www.imdb.com/title/tt0121766/)   | 0.907  |
| [Star Wars: Episode VIII - The Last Jedi (2017)](https://www.imdb.com/title/tt2527336/)        | 0.904  |
| [Star Wars: Episode IX - The Rise of Skywalker (2019)](https://www.imdb.com/title/tt2527338/)  | 0.903  |
| [Star Wars: Episode I - The Phantom Menace (1999)](https://www.imdb.com/title/tt0120915/)      | 0.898  |
| [Star Wars: Episode VII - 

## Functionalize


In [10]:
def related_movies(movie_id, k=10):
    movie = df.filter(pl.col("tconst") == movie_id).rows(named="true")[0]
    query_embed = np.array(movie["embedding"])

    dot_products = query_embed @ embeddings.T
    idx = np.argpartition(dot_products, -k)[-k:]
    idx = idx[np.argsort(dot_products[idx])[::-1]]

    scores = dot_products[idx]

    df_related = df[idx].with_columns(cossim=scores)

    df_md = df_related.head(10).select(
        pl.format(
            "[{} ({})](https://www.imdb.com/title/{}/)", "title", "startYear", "tconst"
        ).alias("title"),
        pl.col("cossim").round(3),
    )

    with pl.Config() as cfg:
        cfg.set_tbl_formatting("ASCII_MARKDOWN")
        cfg.set_fmt_str_lengths(10**5)
        cfg.set_tbl_width_chars(-1)
        cfg.set_tbl_hide_column_data_types(True)
        cfg.set_tbl_hide_dataframe_shape(True)
        print(df_md)


In [11]:
# Sonic the Hedgehog 3
related_movies("tt18259086")

| title                                                                 | cossim |
|-----------------------------------------------------------------------|--------|
| [Sonic the Hedgehog 3 (2024)](https://www.imdb.com/title/tt18259086/) | 1.0    |
| [Sonic the Hedgehog (2020)](https://www.imdb.com/title/tt3794354/)    | 0.927  |
| [Sonic the Hedgehog 2 (2022)](https://www.imdb.com/title/tt12412888/) | 0.914  |
| [The Hedgehog (2024)](https://www.imdb.com/title/tt17080850/)         | 0.894  |
| [The Hedgehog (2009)](https://www.imdb.com/title/tt1442519/)          | 0.845  |
| [The Evil Three (2024)](https://www.imdb.com/title/tt19757976/)       | 0.844  |
| [Hitpig (2024)](https://www.imdb.com/title/tt13176330/)               | 0.843  |
| [Storm Crashers (2024)](https://www.imdb.com/title/tt8816708/)        | 0.838  |
| [Despicable Me 4 (2024)](https://www.imdb.com/title/tt7510222/)       | 0.838  |
| [Shell (2024)](https://www.imdb.com/title/tt12706836/)                | 0.836  |


In [12]:
# Titanic
related_movies("tt0120338")

| title                                                                                      | cossim |
|--------------------------------------------------------------------------------------------|--------|
| [Titanic (1997)](https://www.imdb.com/title/tt0120338/)                                    | 1.0    |
| [Titanic (1953)](https://www.imdb.com/title/tt0046435/)                                    | 0.932  |
| [Titanic (1984)](https://www.imdb.com/title/tt0143942/)                                    | 0.926  |
| [Titanic: 100 Years On (2012)](https://www.imdb.com/title/tt2169432/)                      | 0.926  |
| [Titanic (1943)](https://www.imdb.com/title/tt0036443/)                                    | 0.923  |
| [Titanic (1915)](https://www.imdb.com/title/tt0155274/)                                    | 0.918  |
| [Titanic: The Final Word with James Cameron (2012)](https://www.imdb.com/title/tt2132504/) | 0.914  |
| [Titanic: 20 Years Later with James Cameron (2017)](https://ww

In [13]:
# The Matrix
related_movies("tt0133093")

| title                                                                         | cossim |
|-------------------------------------------------------------------------------|--------|
| [The Matrix (1999)](https://www.imdb.com/title/tt0133093/)                    | 1.0    |
| [The Matrix Reloaded (2003)](https://www.imdb.com/title/tt0234215/)           | 0.937  |
| [The Matrix Revolutions (2003)](https://www.imdb.com/title/tt0242653/)        | 0.929  |
| [The Matrix Resurrections (2021)](https://www.imdb.com/title/tt10838180/)     | 0.926  |
| [The Matrix: The Movie Special (1999)](https://www.imdb.com/title/tt0438231/) | 0.92   |
| [The Matrix: Reborn (2020)](https://www.imdb.com/title/tt12355912/)           | 0.887  |
| [Matrix (2020)](https://www.imdb.com/title/tt11749868/)                       | 0.872  |
| [Jupiter Ascending (2015)](https://www.imdb.com/title/tt1617661/)             | 0.86   |
| [The Living Matrix (2009)](https://www.imdb.com/title/tt1499960/)             | 0.857  |

In [14]:
# Spirited Away
related_movies("tt0245429")

| title                                                                                 | cossim |
|---------------------------------------------------------------------------------------|--------|
| [Spirited Away (2001)](https://www.imdb.com/title/tt0245429/)                         | 1.0    |
| [Spirited Away: Live on Stage (2022)](https://www.imdb.com/title/tt26678995/)         | 0.893  |
| [One Piece: Clockwork Island Adventure (2001)](https://www.imdb.com/title/tt0832449/) | 0.89   |
| [Princess Mononoke (1997)](https://www.imdb.com/title/tt0119698/)                     | 0.89   |
| [Inugami (2001)](https://www.imdb.com/title/tt0277044/)                               | 0.889  |
| [Princess Arete (2001)](https://www.imdb.com/title/tt0306474/)                        | 0.886  |
| [Satorare (2001)](https://www.imdb.com/title/tt0326107/)                              | 0.884  |
| [Harmful Insect (2001)](https://www.imdb.com/title/tt0297905/)                        | 0.883  |
| [Family 

In [15]:
# Everything Everywhere All at Once
related_movies("tt6710474")

| title                                                                             | cossim |
|-----------------------------------------------------------------------------------|--------|
| [Everything Everywhere All at Once (2022)](https://www.imdb.com/title/tt6710474/) | 1.0    |
| [Chilli Laugh Story (2022)](https://www.imdb.com/title/tt17044664/)               | 0.885  |
| [Everything and Everyone (2011)](https://www.imdb.com/title/tt1890403/)           | 0.884  |
| [Baking All the Way (2022)](https://www.imdb.com/title/tt14667790/)               | 0.871  |
| [Everywhere (2010)](https://www.imdb.com/title/tt1744595/)                        | 0.871  |
| [Everything Under Control (2023)](https://www.imdb.com/title/tt25433990/)         | 0.869  |
| [East Bay (2022)](https://www.imdb.com/title/tt3846472/)                          | 0.869  |
| [It Snows All the Time (2022)](https://www.imdb.com/title/tt2767880/)             | 0.869  |
| [Rip in Time (2022)](https://www.imdb.com/title/

In [16]:
# The Avengers
related_movies("tt0848228")

| title                                                                              | cossim |
|------------------------------------------------------------------------------------|--------|
| [The Avengers (2012)](https://www.imdb.com/title/tt0848228/)                       | 1.0    |
| [The Avengers Assemble Premiere (2012)](https://www.imdb.com/title/tt10075836/)    | 0.931  |
| [The Avengers (1998)](https://www.imdb.com/title/tt0118661/)                       | 0.931  |
| [Avengers: Age of Ultron (2015)](https://www.imdb.com/title/tt2395427/)            | 0.908  |
| [Iron Man 2 (2010)](https://www.imdb.com/title/tt1228705/)                         | 0.903  |
| [Avengers: Endgame (2019)](https://www.imdb.com/title/tt4154796/)                  | 0.896  |
| [Captain America: The First Avenger (2011)](https://www.imdb.com/title/tt0458339/) | 0.895  |
| [The Avengers (1950)](https://www.imdb.com/title/tt0167779/)                       | 0.895  |
| [Avengers: Infinity War (2018)](https:

In [17]:
# Avengers: Endgame
related_movies("tt4154796")

| title                                                                      | cossim |
|----------------------------------------------------------------------------|--------|
| [Avengers: Endgame (2019)](https://www.imdb.com/title/tt4154796/)          | 1.0    |
| [Avengers: Infinity War (2018)](https://www.imdb.com/title/tt4154756/)     | 0.909  |
| [The Avengers (2012)](https://www.imdb.com/title/tt0848228/)               | 0.896  |
| [Endgame (2009)](https://www.imdb.com/title/tt1217616/)                    | 0.894  |
| [Captain Marvel (2019)](https://www.imdb.com/title/tt4154664/)             | 0.89   |
| [Avengers: Age of Ultron (2015)](https://www.imdb.com/title/tt2395427/)    | 0.882  |
| [Captain America: Civil War (2016)](https://www.imdb.com/title/tt3498820/) | 0.882  |
| [Endgame (2001)](https://www.imdb.com/title/tt0292502/)                    | 0.881  |
| [The Avengers (1998)](https://www.imdb.com/title/tt0118661/)               | 0.877  |
| [Iron Man 2 (2010)](https://ww

In [18]:
# Thunderbolts*
related_movies("tt20969586")

| title                                                                           | cossim |
|---------------------------------------------------------------------------------|--------|
| [Thunderbolts* (2025)](https://www.imdb.com/title/tt20969586/)                  | 1.0    |
| [Thunderbird (2021)](https://www.imdb.com/title/tt7993812/)                     | 0.864  |
| [Day of Reckoning (2025)](https://www.imdb.com/title/tt35317668/)               | 0.86   |
| [Shadow Force (2025)](https://www.imdb.com/title/tt11092020/)                   | 0.86   |
| [Thunder Force (2021)](https://www.imdb.com/title/tt10121392/)                  | 0.855  |
| [The Unlock (2025)](https://www.imdb.com/title/tt22889888/)                     | 0.85   |
| [The Wizard of the Emerald City (2025)](https://www.imdb.com/title/tt24218194/) | 0.848  |
| [Mickey 17 (2025)](https://www.imdb.com/title/tt12299608/)                      | 0.847  |
| [Thunderbolt (1995)](https://www.imdb.com/title/tt0114126/)         

In [19]:
# Frozen
related_movies("tt2294629")

| title                                                                                                 | cossim |
|-------------------------------------------------------------------------------------------------------|--------|
| [Frozen (2013)](https://www.imdb.com/title/tt2294629/)                                                | 1.0    |
| [Frozen II (2019)](https://www.imdb.com/title/tt4520988/)                                             | 0.93   |
| [Frozen (2010)](https://www.imdb.com/title/tt1323045/)                                                | 0.92   |
| [Frozen (2010)](https://www.imdb.com/title/tt1611845/)                                                | 0.917  |
| [Frozen (1996)](https://www.imdb.com/title/tt0125279/)                                                | 0.909  |
| [Frozen (2005)](https://www.imdb.com/title/tt0376606/)                                                | 0.9    |
| [The Frozen (2012)](https://www.imdb.com/title/tt2363439/)                    

In [20]:
# Lord of the Rings
related_movies("tt0120737")

| title                                                                                                                                      | cossim |
|--------------------------------------------------------------------------------------------------------------------------------------------|--------|
| [The Lord of the Rings: The Fellowship of the Ring (2001)](https://www.imdb.com/title/tt0120737/)                                          | 1.0    |
| [The Lord of the Rings: The Two Towers (2002)](https://www.imdb.com/title/tt0167261/)                                                      | 0.922  |
| [The Lord of the Rings: The Return of the King (2003)](https://www.imdb.com/title/tt0167260/)                                              | 0.92   |
| [National Geographic: Beyond the Movie - The Lord of the Rings: The Fellowship of the Ring (2001)](https://www.imdb.com/title/tt10127200/) | 0.915  |
| [A Passage to Middle-earth: The Making of 'Lord of the Rings' (2001)](https://www.imdb